In [193]:
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np

In [402]:
sheet1 = pd.read_excel("Sample Data for Analysis.xlsx", "Raw Data")
sheet2 = pd.read_excel("Sample Data for Analysis.xlsx", "Distances to Campus")
sheet3 = pd.read_excel("Sample Data for Analysis.xlsx", "Campus Key")
sheet4 = pd.read_excel("Sample Data for Analysis (1).xlsx", "Online Students")
sheet5 = pd.read_csv("Sample Data for Analysis.xlsx - Combined.csv")

In [345]:
sheet1

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID
0,15900,WEB,CT,52.0401,"Admin. Assistant, General",SPRINGFIELD,MO,2.907700e+10
1,18411,WEB,CT,52.0401,"Admin. Assistant, General",WAYNESVILLE,MO,2.916947e+10
2,18533,WEB,CT,52.0401,"Admin. Assistant, General",SAINT ROBERT,MO,2.916947e+10
3,10847,MNC,AAS,1.0000,"Agriculture, General",WILLARD,MO,2.907701e+10
4,11036,MNC,AAS,1.0000,"Agriculture, General",SEYMOUR,MO,2.922547e+10
...,...,...,...,...,...,...,...,...
18402,17515,WEB,AAS,48.0508,Welding Technology/Welder,ROGERSVILLE,MO,2.922547e+10
18403,17720,WEB,AAS,48.0508,Welding Technology/Welder,WALNUT SHADE,MO,2.921348e+10
18404,17796,WEB,AAS,48.0508,Welding Technology/Welder,MORRIS,IL,1.706300e+10
18405,18162,WEB,AAS,48.0508,Welding Technology/Welder,SPRINGFIELD,MO,2.907700e+10


In [346]:
sheet2

,GEOID,Distance,Campus
0,29105960400,1.93467,East Campus
1,29105960500,2.19978,East Campus
2,29105960300,2.89078,East Campus
3,29105960600,3.53737,East Campus
4,29105960100,9.27136,East Campus
...,...,...,...
15783,13115001600,465.62900,West Campus
15784,28035010102,475.41600,West Campus
15785,46023940300,481.38900,West Campus
15786,1051030400,507.49800,West Campus


In [347]:
sheet3

,Student Campus Abbreviation,Campus Full Name
0,ETC,East Campus
1,MNC,Main Campus
2,WTC,West Campus
3,WEB,Online


In [348]:
# check for nan
sheet1.isnull().values.any()

True

In [349]:
sheet1[sheet1['GEOID'].isnull() == True]

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID
357,7688,MNC,AAS,47.0604,Auto Mechanics Tech,ELKLAND,MO,NaN
877,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",MOUNTAIN GROVE,MO,NaN
1383,17120,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,SPRINGFIELD,MO,NaN
1421,18268,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,STOCKTON,MO,NaN
1767,1521,MNC,AAS,10.0105,Communications Technology/Technician,OZARK,MO,NaN
...,...,...,...,...,...,...,...,...
17599,12979,MNC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",BRANSON,MO,NaN
17622,13275,WTC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",WAYNESVILLE,MO,NaN
17798,15739,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",APO,AE,NaN
17906,17102,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",MOUNTAIN VIEW,MO,NaN


In [350]:
sheet1["STUDENTS"].count()

18407

In [351]:
len(list(sheet1["STUDENTS"].unique()))

18407

In [352]:
sheet1["GEOID"].count()

18280

In [353]:
len(list(sheet1["GEOID"].unique()))

1088

In [354]:
# data2 = sheet1[sheet1['GEOID'].isnull() == False]

In [355]:
dict_list = []
dict_list2 = []
for i in list(sheet1[["City", "State", "GEOID"]]):
    unique = list(sheet1[i].unique())
    num_list = list(range(len(unique)))
    dictionary = dict(zip(unique, num_list))
    reverse = dict(zip(num_list, unique))
    dict_list.append(dictionary)
    dict_list2.append(reverse)

In [356]:
data2 = sheet1[sheet1['GEOID'].isnull() == False]

In [357]:
data2["City"].replace(dict_list[0], inplace=True)
data2["State"].replace(dict_list[1], inplace=True)
data2["GEOID"].replace(dict_list[2], inplace=True)

C:\Users\Stefhon.JeanClaude\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [358]:
X_train = data2[["City", "State"]][0:14000]
y_train = data2["GEOID"][0:14000]

In [359]:
X_test = data2[["City", "State"]][14000:16000]
y_test = data2["GEOID"][14000:16000]

In [360]:
feature_scores_id = []
feature_scores = []
f_dict = []

# X_train = np.array(X_train)
# X_test = np.array(X_test)
# X_test = X_test.reshape(-1,1)

model = RandomForestClassifier()
model.fit(X_train, y_train)

importance = model.feature_importances_
for i,v in enumerate(importance):
    # print('Feature: %0d, Score: %.11f' % (i,v))
    feature_scores_id.append(i)
    feature_scores.append(v)

In [361]:
y_pred = list(model.predict(X_test))

In [362]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
accuracy

0.3635

In [363]:
data3 = sheet1[sheet1['GEOID'].isnull() == True]
data3["City"].replace(dict_list[0], inplace=True)
data3["State"].replace(dict_list[1], inplace=True)
data3["GEOID"].replace(dict_list[2], inplace=True)

# X_train = data3[["City", "State"]][0:50]
# y_train = data3["GEOID"][0:50]

X_test = data3[["City", "State"]]
y_test = data3["GEOID"]

C:\Users\Stefhon.JeanClaude\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [364]:
y_pred = list(model.predict(X_test))

In [365]:
y_pred = pd.DataFrame(y_pred)

In [366]:
y_pred.replace(dict_list2[2], inplace=True)

In [367]:
y_pred

,0
0,2.922547e+10
1,2.922949e+10
2,2.907700e+10
3,2.903987e+10
4,2.904302e+10
...,...
122,2.921348e+10
123,2.916947e+10
124,4.602394e+10
125,2.909109e+10


In [368]:
y_pred.to_csv("GEOID.csv")

In [ ]:
##

In [396]:
distance_list = []
for i in sheet4["GEOID"]:
    try:
        d = sheet2[sheet2["GEOID"] == i]
        list1 = list(d["Distance"])
        list1.sort()
        min_val = list1[0]
        distance_list.append(min_val)
    except:
        distance_list.append(None)

In [399]:
dist = pd.DataFrame(distance_list)

In [400]:
dist.to_csv("online_students_distance.csv")

In [ ]:
##

In [417]:
list(sheet5["Distance"])

[14.444,
 28.9428,
 14.8487,
 8.59492,
 10.234,
 40.4224,
 182.166,
 19.1679,
 9.13251,
 2.23669,
 5.7841,
 3.19082,
 6.95345,
 11.9212,
 3.44009,
 6.95345,
 6.95345,
 4.46439,
 6.77106,
 7.46766,
 23.8286,
 3.53722,
 14.5007,
 11.2301,
 3.0433,
 9.61334,
 6.77106,
 13.3667,
 45.1876,
 21.0613,
 19.9713,
 33.9573,
 1.07272,
 14.8487,
 3.10538,
 4.08108,
 14.8487,
 57.1356,
 3.19082,
 14.8487,
 81.6754,
 37.0125,
 23.279,
 5.7841,
 14.5007,
 8.59492,
 32.7724,
 14.9602,
 8.59492,
 10.234,
 3.56542,
 7.46766,
 1.84448,
 7.46766,
 1.34949,
 30.4661,
 4.20579,
 17.0655,
 108.322,
 14.0589,
 6.70902,
 1.84448,
 14.5007,
 19.1679,
 10.6912,
 13.8544,
 14.0589,
 18.4473,
 188.885,
 29.9591,
 3.0433,
 24.5586,
 56.9376,
 9.13251,
 65.169,
 2.14265,
 14.9602,
 14.4019,
 7.35952,
 0.94197,
 4.08108,
 38.2414,
 12.0382,
 1.34949,
 50.2859,
 14.5007,
 3.06891,
 14.9602,
 65.8478,
 4.08108,
 9.13251,
 29.9591,
 12.5947,
 214.143,
 38.2414,
 42.3628,
 42.3628,
 82.8305,
 4.69276,
 58.4632,
 63.804,


In [424]:
for i in list(range(0,301, 20)):
    print(f'''elif {i} < i <= {i+20}:
        bucket.append("{i}-{i+20}")''')

elif 0 < i <= 20:
        bucket.append("0-20")
elif 20 < i <= 40:
        bucket.append("20-40")
elif 40 < i <= 60:
        bucket.append("40-60")
elif 60 < i <= 80:
        bucket.append("60-80")
elif 80 < i <= 100:
        bucket.append("80-100")
elif 100 < i <= 120:
        bucket.append("100-120")
elif 120 < i <= 140:
        bucket.append("120-140")
elif 140 < i <= 160:
        bucket.append("140-160")
elif 160 < i <= 180:
        bucket.append("160-180")
elif 180 < i <= 200:
        bucket.append("180-200")
elif 200 < i <= 220:
        bucket.append("200-220")
elif 220 < i <= 240:
        bucket.append("220-240")
elif 240 < i <= 260:
        bucket.append("240-260")
elif 260 < i <= 280:
        bucket.append("260-280")
elif 280 < i <= 300:
        bucket.append("280-300")
elif 300 < i <= 320:
        bucket.append("300-320")


In [425]:
bucket = []
for i in sheet5["Distance"]:
    if i <=20:
        bucket.append("0-20")
    elif 20 < i <= 40:
        bucket.append("20-40")
    elif 40 < i <= 60:
            bucket.append("40-60")
    elif 60 < i <= 80:
            bucket.append("60-80")
    elif 80 < i <= 100:
            bucket.append("80-100")
    elif 100 < i <= 120:
            bucket.append("100-120")
    elif 120 < i <= 140:
            bucket.append("120-140")
    elif 140 < i <= 160:
            bucket.append("140-160")
    elif 160 < i <= 180:
            bucket.append("160-180")
    elif 180 < i <= 200:
            bucket.append("180-200")
    elif 200 < i <= 220:
            bucket.append("200-220")
    elif 220 < i <= 240:
            bucket.append("220-240")
    elif 240 < i <= 260:
            bucket.append("240-260")
    elif 260 < i <= 280:
            bucket.append("260-280")
    elif 280 < i <= 300:
            bucket.append("280-300")
    else:
        bucket.append(">300")
    
    

In [426]:
bucket

['0-20',
 '20-40',
 '0-20',
 '0-20',
 '0-20',
 '40-60',
 '180-200',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '20-40',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '40-60',
 '20-40',
 '0-20',
 '20-40',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '40-60',
 '0-20',
 '0-20',
 '80-100',
 '20-40',
 '20-40',
 '0-20',
 '0-20',
 '0-20',
 '20-40',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '20-40',
 '0-20',
 '0-20',
 '100-120',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '180-200',
 '20-40',
 '0-20',
 '20-40',
 '40-60',
 '0-20',
 '60-80',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '0-20',
 '20-40',
 '0-20',
 '0-20',
 '40-60',
 '0-20',
 '0-20',
 '0-20',
 '60-80',
 '0-20',
 '0-20',
 '20-40',
 '0-20',
 '200-220',
 '20-40',
 '40-60',
 '40-60',
 '80-100',
 '0-20',
 '40-60',
 '60-80',
 '0-20',
 '0-20',
 '0-20',
 '180-200',
 '0-20',
 '0

In [427]:
sheet5["Dist Bucket"] = bucket

In [428]:
sheet5

,STUDENTS,GEOID,Distance,Campus Name,Dist Bucket
0,10847,29077005002,14.44400,Main Campus,0-20
1,11036,29225470401,28.94280,Main Campus,20-40
2,11248,29043020102,14.84870,Main Campus,0-20
3,11676,29077004101,8.59492,Main Campus,0-20
4,12289,29043020202,10.23400,Main Campus,0-20
...,...,...,...,...,...
18402,17515,29225470301,14.96020,Online,0-20
18403,17720,29213480202,31.86450,Online,20-40
18404,17796,17063000500,314.45200,Online,>300
18405,18162,29077001800,1.86673,Online,0-20


In [430]:
sheet5.to_csv("bucketed_data.csv")

In [ ]:
############################################################################################################################

In [306]:
data3 = data3.reset_index(drop=True)

In [307]:
data3

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID
0,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,145.0
1,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50,0,145.0
2,17120,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,0,0,145.0
3,18268,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,94,0,145.0
4,1521,MNC,AAS,10.0105,Communications Technology/Technician,18,0,145.0
...,...,...,...,...,...,...,...,...
122,12979,MNC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",7,0,145.0
123,13275,WTC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",1,0,145.0
124,15739,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",693,43,145.0
125,17102,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",203,0,145.0


In [308]:
data3["GEOID"] = y_pred

In [309]:
data3

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID
0,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,2.922547e+10
1,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50,0,2.922949e+10
2,17120,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,0,0,2.907700e+10
3,18268,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,94,0,2.903987e+10
4,1521,MNC,AAS,10.0105,Communications Technology/Technician,18,0,2.904302e+10
...,...,...,...,...,...,...,...,...
122,12979,MNC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",7,0,2.921348e+10
123,13275,WTC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",1,0,2.916947e+10
124,15739,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",693,43,4.602394e+10
125,17102,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",203,0,2.909109e+10


In [311]:
new_data = pd.concat([data3, data2], ignore_index=True)
new_data

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID
0,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,2.922547e+10
1,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50,0,2.922949e+10
2,17120,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,0,0,2.907700e+10
3,18268,WEB,AAS,52.0302,Bookkeeping and Accounting Tech,94,0,2.903987e+10
4,1521,MNC,AAS,10.0105,Communications Technology/Technician,18,0,2.904302e+10
...,...,...,...,...,...,...,...,...
18402,17515,WEB,AAS,48.0508,Welding Technology/Welder,21,0,4.900000e+01
18403,17720,WEB,AAS,48.0508,Welding Technology/Welder,82,0,1.380000e+02
18404,17796,WEB,AAS,48.0508,Welding Technology/Welder,799,7,1.087000e+03
18405,18162,WEB,AAS,48.0508,Welding Technology/Welder,0,0,1.100000e+02


In [326]:
new_data2 = new_data.merge(sheet2, on=["GEOID"], how="inner")

In [327]:
new_data2

,STUDENTS,Campus_x,Degree Level,CIP Code,Program Title,City,State,GEOID,Distance,Campus_y
0,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,2.922547e+10,29.7163,East Campus
1,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,2.922547e+10,19.4975,Main Campus
2,7688,MNC,AAS,47.0604,Auto Mechanics Tech,89,0,2.922547e+10,51.8064,West Campus
3,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50,0,2.922949e+10,44.3244,East Campus
4,9416,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50,0,2.922949e+10,53.4675,Main Campus
...,...,...,...,...,...,...,...,...,...,...
376,605,ETC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",84,0,2.922949e+10,40.6923,Main Campus
377,605,ETC,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",84,0,2.922949e+10,42.6853,West Campus
378,17102,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",203,0,2.909109e+10,68.4060,East Campus
379,17102,WEB,AAT,13.1399,"Teacher Ed., Spec. Subjects, Other",203,0,2.909109e+10,86.2565,Main Campus


In [332]:
for i in list(sheet1["GEOID"].unique()):
    if i not in list(sheet2["GEOID"].unique()):
        print("\n\nNOT IN LIST\n\n")
    else:
        print("in")

in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


NOT IN LIST


in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


NOT IN LIST


in
in
in
in
in
in
in
in
in
in


NOT IN LIST




NOT IN LIST




NOT IN LIST


in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


NOT IN LIST


in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


NOT IN LIST


in
in
in
in


NOT IN LIST


in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


NOT IN LIST




NOT IN LIST


in
in


NOT IN LIST


in
in
in
in
in
in
in


NOT IN LIST




NOT IN LIST


in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


In [333]:
new_data2 = new_data.merge(sheet2, on=["GEOID"], how="outer")

In [334]:
new_data2

,STUDENTS,Campus_x,Degree Level,CIP Code,Program Title,City,State,GEOID,Distance,Campus_y
0,7688.0,MNC,AAS,47.0604,Auto Mechanics Tech,89.0,0.0,2.922547e+10,29.7163,East Campus
1,7688.0,MNC,AAS,47.0604,Auto Mechanics Tech,89.0,0.0,2.922547e+10,19.4975,Main Campus
2,7688.0,MNC,AAS,47.0604,Auto Mechanics Tech,89.0,0.0,2.922547e+10,51.8064,West Campus
3,9416.0,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50.0,0.0,2.922949e+10,44.3244,East Campus
4,9416.0,MNC,AS,26.0101,"Biology/Biological Sciences, Gen'l",50.0,0.0,2.922949e+10,53.4675,Main Campus
...,...,...,...,...,...,...,...,...,...,...
34300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.812103e+10,299.7970,West Campus
34301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.910507e+10,299.8580,West Campus
34302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.816195e+10,299.9750,West Campus
34303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.051030e+09,507.4980,West Campus


In [339]:
new_data2 = new_data2[new_data2["STUDENTS"] >= 0]

In [342]:
for i in list(range(len(new_data2))):
    dist = new_data2["Distance"].iloc[i]
    print(dist)

29.7163
19.4975
51.8064
44.3244
53.4675
48.9819
44.3244
53.4675
48.9819
44.3244
53.4675
48.9819
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345
76.8328
54.9797
6.95345


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
